# Run MATLAB Vectorization

This interactive notebook allows you to run **only** the MATLAB vectorization pipeline. Configured with interactive widgets for ease of use.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import json
import os
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root to path
project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

from slavv.dev.comparison import load_parameters, orchestrate_comparison

## Configuration

Use the widgets below to configure the MATLAB run.

In [ ]:
# Find available TIFF files
search_paths = [
    project_root / "data",
    project_root / "tests" / "data"
]
available_files = []
for p in search_paths:
    if p.exists():
        available_files.extend([str(f) for f in p.glob("*.tif")])
        available_files.extend([str(f) for f in p.glob("*.tiff")])

# Defaults
default_input = available_files[0] if available_files else ""
default_matlab = "C:\\Program Files\\MATLAB\\R2019a\\bin\\matlab.exe"
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
default_output = str(project_root / "comparisons" / f"{timestamp}_matlab_run")

# Styles
section_header_style = "font-size: 14px; font-weight: bold; color: #2c3e50; margin-bottom: 5px; border-bottom: 2px solid #eaeaea; padding-bottom: 5px; width: 100%"
box_layout = widgets.Layout(
    border='1px solid #e0e0e0', 
    padding='15px', 
    margin='10px 0', 
    border_radius='8px',
    width='95%'
)

def create_header(text):
    return widgets.HTML(f"<div style='{section_header_style}'>{text}</div>")

# Widgets
# File Selection
input_widget = widgets.Combobox(
    options=available_files,
    value=default_input,
    description='Input File:',
    placeholder='Type path or select from list',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

file_box = widgets.VBox([
    create_header("File Input"),
    input_widget
], layout=box_layout)

# Path Configuration
matlab_widget = widgets.Text(
    value=default_matlab,
    description='MATLAB Path:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

output_widget = widgets.Text(
    value=default_output,
    description='Output Dir:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

path_box = widgets.VBox([
    create_header("System Paths"),
    matlab_widget,
    output_widget
], layout=box_layout)

display(widgets.VBox([file_box, path_box]))

## Run Pipeline

In [ ]:
run_button = widgets.Button(
    description='Start MATLAB Vectorization',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='100%', height='50px'),
    icon='play-circle', 
    style={'font_weight': 'bold', 'font_size': '16px'}
)

output_area = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='20px 0',
        max_height='400px',
        overflow='scroll'
    )
)

def on_run_clicked(b):
    # Get values from widgets
    input_file = input_widget.value
    output_dir = Path(output_widget.value)
    matlab_path = matlab_widget.value
    
    params = load_parameters(str(project_root / "scripts" / "comparison_params.json"))

    run_button.disabled = True
    run_button.description = "Running MATLAB..."
    
    with output_area:
        clear_output()
        print(f"Starting MATLAB run at {datetime.now().strftime('%H:%M:%S')}...")
        print("-" * 40)
        
        if not os.path.exists(input_file):
            print(f"Error: Input file not found: {input_file}")
            run_button.disabled = False
            run_button.description = 'Start MATLAB Vectorization'
            return

        # Create output directory
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Run with skip_python=True
        try:
            status = orchestrate_comparison(
                input_file=input_file,
                output_dir=output_dir,
                matlab_path=matlab_path,
                project_root=project_root,
                params=params,
                skip_matlab=False,
                skip_python=True
            )
            
            print("-" * 40)
            if status == 0:
                print("\nMATLAB vectorization completed successfully!")
            else:
                print("\nMATLAB vectorization failed. Check logs above.")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()
        finally:
            run_button.disabled = False
            run_button.description = 'Start MATLAB Vectorization'

run_button.on_click(on_run_clicked)

display(widgets.VBox([run_button, output_area]))

## Next Steps

Check the output directory for MATLAB results (in `matlab_results/` folder).

Then, open **`03_Compare_Results.ipynb`** to compare with Python results.